In [1]:
# pip install pandas

In [2]:
# pip install pymysql

In [3]:
from pyspark.sql import SparkSession
from pyspark import SparkContext
import pyspark
import pandas as pd
import pymysql

----- Extract -----

In [4]:
sc = pyspark.SparkContext('local[*]')

In [5]:
# sc.stop()
spark = SparkSession.\
        builder.\
        appName("hello pyspark").\
        master("spark://spark-master:7077").\
        config("spark.executor.memory", "512m").\
        getOrCreate()

In [6]:
df = spark.read.csv(path="default of credit card clients.csv", sep=",", header=True).cache()

In [7]:
df.show()

+---+---------+---+---------+--------+---+-----+-----+-----+-----+-----+-----+---------+---------+---------+---------+---------+---------+--------+--------+--------+--------+--------+--------+--------------------------+
| ID|LIMIT_BAL|SEX|EDUCATION|MARRIAGE|AGE|PAY_0|PAY_2|PAY_3|PAY_4|PAY_5|PAY_6|BILL_AMT1|BILL_AMT2|BILL_AMT3|BILL_AMT4|BILL_AMT5|BILL_AMT6|PAY_AMT1|PAY_AMT2|PAY_AMT3|PAY_AMT4|PAY_AMT5|PAY_AMT6|default payment next month|
+---+---------+---+---------+--------+---+-----+-----+-----+-----+-----+-----+---------+---------+---------+---------+---------+---------+--------+--------+--------+--------+--------+--------+--------------------------+
|  1|    20000|  2|        2|       1| 24|    2|    2|   -1|   -1|   -2|   -2|     3913|     3102|      689|        0|        0|        0|       0|     689|       0|       0|       0|       0|                         1|
|  2|   120000|  2|        2|       2| 26|   -1|    2|    0|    0|    0|    2|     2682|     1725|     2682|     3272|  

In [8]:
df.printSchema()

root
 |-- ID: string (nullable = true)
 |-- LIMIT_BAL: string (nullable = true)
 |-- SEX: string (nullable = true)
 |-- EDUCATION: string (nullable = true)
 |-- MARRIAGE: string (nullable = true)
 |-- AGE: string (nullable = true)
 |-- PAY_0: string (nullable = true)
 |-- PAY_2: string (nullable = true)
 |-- PAY_3: string (nullable = true)
 |-- PAY_4: string (nullable = true)
 |-- PAY_5: string (nullable = true)
 |-- PAY_6: string (nullable = true)
 |-- BILL_AMT1: string (nullable = true)
 |-- BILL_AMT2: string (nullable = true)
 |-- BILL_AMT3: string (nullable = true)
 |-- BILL_AMT4: string (nullable = true)
 |-- BILL_AMT5: string (nullable = true)
 |-- BILL_AMT6: string (nullable = true)
 |-- PAY_AMT1: string (nullable = true)
 |-- PAY_AMT2: string (nullable = true)
 |-- PAY_AMT3: string (nullable = true)
 |-- PAY_AMT4: string (nullable = true)
 |-- PAY_AMT5: string (nullable = true)
 |-- PAY_AMT6: string (nullable = true)
 |-- default payment next month: string (nullable = true)



In [9]:
df.count()

30000

----- Transform -----

In [10]:
df = df.drop("PAY_0","PAY_2","PAY_3","PAY_4","PAY_5","PAY_6")

In [11]:
df = df.drop("BILL_AMT1","BILL_AMT2","BILL_AMT3","BILL_AMT4","BILL_AMT5","BILL_AMT6")

In [12]:
df = df.drop("PAY_AMT1","PAY_AMT2","PAY_AMT3","PAY_AMT4","PAY_AMT5","PAY_AMT6")

In [13]:
df.describe().show()

+-------+-----------------+------------------+------------------+------------------+------------------+-----------------+--------------------------+
|summary|               ID|         LIMIT_BAL|               SEX|         EDUCATION|          MARRIAGE|              AGE|default payment next month|
+-------+-----------------+------------------+------------------+------------------+------------------+-----------------+--------------------------+
|  count|            30000|             30000|             30000|             30000|             30000|            30000|                     30000|
|   mean|          15000.5|167484.32266666667|1.6037333333333332|1.8531333333333333|1.5518666666666667|          35.4855|                    0.2212|
| stddev|8660.398374208891|129747.66156720246|0.4891291960902602|0.7903486597207269|0.5219696006132467|9.217904068090155|       0.41506180569093254|
|    min|                1|             10000|                 1|                 0|                 0|   

In [14]:
df.show()

+---+---------+---+---------+--------+---+--------------------------+
| ID|LIMIT_BAL|SEX|EDUCATION|MARRIAGE|AGE|default payment next month|
+---+---------+---+---------+--------+---+--------------------------+
|  1|    20000|  2|        2|       1| 24|                         1|
|  2|   120000|  2|        2|       2| 26|                         1|
|  3|    90000|  2|        2|       2| 34|                         0|
|  4|    50000|  2|        2|       1| 37|                         0|
|  5|    50000|  1|        2|       1| 57|                         0|
|  6|    50000|  1|        1|       2| 37|                         0|
|  7|   500000|  1|        1|       2| 29|                         0|
|  8|   100000|  2|        2|       2| 23|                         0|
|  9|   140000|  2|        3|       1| 28|                         0|
| 10|    20000|  1|        3|       2| 35|                         0|
| 11|   200000|  2|        3|       2| 34|                         0|
| 12|   260000|  2| 

In [15]:
for col in df.columns:
    print(df.filter(df[col].isNull()).count())

0
0
0
0
0
0
0


In [16]:
df.createOrReplaceTempView("clients")

In [17]:
spark.sql("SELECT count(*) FROM clients").show()

+--------+
|count(1)|
+--------+
|   30000|
+--------+



In [18]:
df= spark.sql("SELECT * FROM clients").toPandas()

In [20]:
df.columns

Index(['ID', 'LIMIT_BAL', 'SEX', 'EDUCATION', 'MARRIAGE', 'AGE',
       'default payment next month'],
      dtype='object')

In [21]:
df.head()

,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,default payment next month
0,1,20000,2,2,1,24,1
1,2,120000,2,2,2,26,1
2,3,90000,2,2,2,34,0
3,4,50000,2,2,1,37,0
4,5,50000,1,2,1,57,0


In [22]:
df['SEX'].replace({'1':'Male','2':'Female'}, inplace=True)

In [23]:
df['EDUCATION'].replace({'1':'Graduate school','2':'University','3':'High school','4':'others','5':'unknown','6':'unknown','0':'unknown'}, inplace=True)

In [24]:
df['MARRIAGE'].replace({'1':'Married','2':'Single','3':'others','0':'unknown'}, inplace=True)

In [25]:
df['default payment next month'].replace({'1':'Yes','0':'No'}, inplace=True)

In [26]:
df.tail()

,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,default payment next month
29995,29996,220000,Male,High school,Married,39,No
29996,29997,150000,Male,High school,Single,43,No
29997,29998,30000,Male,University,Single,37,Yes
29998,29999,80000,Male,High school,Married,41,Yes
29999,30000,50000,Male,University,Married,46,Yes


----- Load -----

In [27]:
from sqlalchemy import create_engine

In [28]:
username = "root"
password = "dstb"
port = 3306
database = "dstb_db"

In [29]:
pymysql.install_as_MySQLdb()

In [30]:
engine = create_engine('mysql+mysqldb://%s:%s@db:%i/%s'%(username, password, port, database))

In [31]:
df.to_sql("clients", engine, if_exists="replace")

In [32]:
%load_ext sql

In [33]:
%sql mysql://root:dstb@db:3306/dstb_db

In [34]:
result = %sql select * from clients

 * mysql://root:***@db:3306/dstb_db
30000 rows affected.


In [35]:
%sql select * from clients limit 5

 * mysql://root:***@db:3306/dstb_db
5 rows affected.


index,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,default payment next month
0,1,20000,Female,University,Married,24,Yes
1,2,120000,Female,University,Single,26,Yes
2,3,90000,Female,University,Single,34,No
3,4,50000,Female,University,Married,37,No
4,5,50000,Male,University,Married,57,No


In [36]:
result = %sql select * from clients limit 5

 * mysql://root:***@db:3306/dstb_db
5 rows affected.


In [37]:
result

index,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,default payment next month
0,1,20000,Female,University,Married,24,Yes
1,2,120000,Female,University,Single,26,Yes
2,3,90000,Female,University,Single,34,No
3,4,50000,Female,University,Married,37,No
4,5,50000,Male,University,Married,57,No


In [38]:
type(result)

sql.run.ResultSet

In [39]:
result = result.DataFrame()

In [40]:
result.head()

,index,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,default payment next month
0,0,1,20000,Female,University,Married,24,Yes
1,1,2,120000,Female,University,Single,26,Yes
2,2,3,90000,Female,University,Single,34,No
3,3,4,50000,Female,University,Married,37,No
4,4,5,50000,Male,University,Married,57,No
